# Freies Teilchen

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird das Zerfließen eines anfänglich am Ursprung zentrierten Gauß'schen Wellenpakets

$$\psi(x) = \frac{1}{(2\pi)^{1/4}}\exp\left(-\frac{x^2}{4}+\text{i}k_0x\right)$$

mit Impuls $k_0$ untersucht. In der dimensionslosen zeitabhängigen Schrödingergleichung für das freie Teilchen

$$\text{i}\frac{\partial}{\partial t}\Psi(x,t) = -\frac{1}{2}\frac{\partial^2}{\partial^2x}\Psi(x,t) + V(x)$$

sehen wir ein optisches Potential

$$V(x) = -\text{i} V_\text{opt} \left[\exp\left(-\frac{(x+x_\text{max})^2}{2\sigma_\text{opt}^2}\right)
                                      + \exp\left(-\frac{(x-x_\text{max})^2}{2\sigma_\text{opt}^2}\right)\right]$$

vor, das durch an den Rändern des Ortsbereich positionierte Gauß-Funktionen variabler Stärke und Breite gegeben ist. Dadurch lassen sich Reflexionen an dem in der numerischen Behandlung begrenzten Ortsbereich unterdrücken. Die Lösung der zeitabhängigen Schrödingergleichung erfolgt  mit Hilfe der Split-Operator-Methode. Dabei verwenden wir den FFT-Algorithmus, um zwischen der Orts- und der Impulsdarstellung zu wechseln. Im ersten Teil des Jupyter-Notebooks wird die zeitliche Dynamik des Wellenpakets bestimmt und mit Hilfe einer Animation dargestellt.

*Hinweis:* Sollte die Animation nicht funktionieren, stellen Sie sicher, dass Sie dieses Jupyter-Notebook in JupyterLab ausführen.

Am Ende dieses Jupyter-Notebooks wird noch die Standardabweichung des Ortes berechnet und mit dem analytisch bekannten Ergebnis

$$\sigma(t) = \sqrt{1+\frac{t^2}{4}}$$

verglichen. Damit lässt sich die Güte der numerischen Lösung für das Zerfließen des Gauß'schen Wellenpakets einschätzen.

## Importanweisungen

Zusätzlich zu bereits bekannten Importanweisungen wird hier das `animation`-Modul aus der matplotlib-Bibliothek importiert, das für die Animation des Zerfließens des Wellenpakets erforderlich ist.

In [ ]:
from math import pi, sqrt
import numpy as np
import numpy.linalg as LA
from scipy import fft
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output, display, HTML
import matplotlib.pyplot as plt
from matplotlib import animation

plt.style.use("numphyspy.style")

## Zerfließen eines Wellenpakets und Einfluss eines optischen Potentials

### Definition des optischen Potentials

Die Berechnung des optischen Potentials gemäß der oben angegebenen Formel lässt für das Argument `x` sowohl skalare Werte als auch NumPy-Arrays zu.

In [ ]:
def v_optical(x, v_opt, sigma_opt, x_max):
    v = -1j*v_opt*(np.exp(-(x+x_max)**2/(2*sigma_opt**2))
                   + np.exp(-(x-x_max)**2/(2*sigma_opt**2)))
    return v

### Transformation vom Orts- in den Impulsraum und zurück

Der Wechsel zwischen Orts- und Impulsraum erfolgt mit Hilfe einer Fourier-Transformation oder ihrer Inversen.

In [ ]:
def position_to_momentum(psi_position):
    psi_momentum = fft.fft(psi_position)
    return psi_momentum

In [ ]:
def momentum_to_position(psi_momentum):
    psi_position = fft.ifft(psi_momentum)
    return psi_position

### Erzeugung des Anfangszustands

Der Anfangszustand ist durch die oben angegebene Gauß-Form gegeben. Die Normierung erfolgt hier nicht durch Verwendung des analytischen Wertes, sondern durch explizite Normierung des Zustandsvektors auf dem Ortsgitter.

In [ ]:
def initial_state(x, dx, k_0):
    psi_position = np.exp(-0.25*x**2 + 1j*k_0*x)
    norm = np.sum(np.absolute(psi_position)**2) * dx
    return psi_position / sqrt(norm)

### Zeitschritt

Um einen einzelnen Zeitschritt im Rahmen der Split-Operator-Methode auszuführen, wird die Wellenfunktion im Ortsraum zunächst mit einem Phasenfaktor multipliziert, der das Potential `v` über die Hälfte des Zeitintervalls `dt` berücksichtigt. Anschließend erfolgt der Wechsel in den Impulsraum, um den Zustandsvektor mit dem zur kinetischen Energie gehörenden Phasenfaktor für das volle Zeitintervall zu multiplizieren. Dabei ist `k` ein NumPy-Array, das die zum gewählten Ortsgitter gehörigen Impulse enthält. Nach dem abschließenden Wechsel zurück in den Ortsraum erfolgt die Multiplikation mit dem Phasenfaktor, der das Potential über die noch fehlende Hälfte des Zeitintervalls berücksichtigt.

In [ ]:
def time_step(psi_position, dt, v, k):
    psi_position = psi_position * np.exp(-1j*v*dt/2)
    psi_momentum = position_to_momentum(psi_position)
    psi_momentum = psi_momentum * np.exp(-1j*k**2/2*dt)
    psi_position = momentum_to_position(psi_momentum)
    psi_position = psi_position * np.exp(-1j*v*dt/2)
    return psi_position

### Lösung der Schrödingergleichung

Nachdem das optische Potential und der Anfangszustand im Ortsraum auf dem vorgegebenen räumlichen Gitter berechnet wurden, wird ein NumPy-Array für die Aufenthaltswahrscheinlichkeit im Ortsraum als Funktion der Zeit angelegt und die erste Zeile mit der Aufenthaltswahrscheinlichkeit des Anfangszustands gefüllt. Außerdem werden die für die Funktion `time_step` benötigten Impulswerte `k` mit Hilfe von `fft.fftfreq` aus den diskreten Ortswerten des räumlichen Gitters berechnet. Damit kann dann entsprechend der Anzahl der in `t_values` vorgegebenen Zeitwerte die schrittweise Berechnung der Zeitentwicklung durch Aufruf der Funktion `time_step` erfolgen.

In [ ]:
def time_development(t_values, dt, x_values, dx, x_max,
                     k_0, v_opt, sigma_opt):
    n_x_points = x_values.shape[0]
    n_t_points = t_values.shape[0]
    v = v_optical(x_values, v_opt, sigma_opt, x_max)
    psi_position = initial_state(x_values, dx, k_0)
    psi_squared_of_t = np.zeros((n_t_points, n_x_points))
    psi_squared_of_t[0, :] = np.abs(psi_position)**2
    k = fft.fftfreq(n_x_points, dx) * 2*pi
    for n in range(n_t_points-1):
        psi_position = time_step(psi_position, dt, v, k)
        psi_squared_of_t[n+1, :] = np.abs(psi_position)**2
    return psi_squared_of_t

### Implementierung des Bedienelements und animierte Darstellung der Aufenthaltswahrscheinlichkeit

Mit Hilfe des Drop-down-Menüs lassen sich vier vorgegebene interessante Parametersätze für das optische Potential auswählen:

| <div style="width:20em"></div>    |<div style="width:7em">$V_\text{opt}$</div>| <div style="width:7em">$\sigma_\text{opt}$</div> |
|:----------------------------------|------------------------------------------:|-------------------------------------------------:|
| Kein optisches Potential          |                                       0   |                                              1   |
| Gut gewähltes optisches Potential |                                      20   |                                              5   |
| Zu schwaches optisches Potential  |                                       2   |                                              2   |
| Zu steiles optisches Potential    |                                  100000   |                                            0.1   |

Die Berechnung der zeitabhängigen Aufenthaltswahrscheinlichkeit kann etwas Zeit in Anspruch nehmen. Erst nach Abschluss der Berechnung erfolgt die Darstellung als Animation. Mit den Steuerelementen kann man die Animation erneut ablaufen lassen, ohne die Daten neu berechnen zu müssen. Der Ablauf lässt sich bei Bedarf auch unterbrechen.

In [ ]:
params_widget = widgets.Dropdown(
    options=[
        ("Kein optisches Potential", (0, 1)),
        ("Gut gewähltes optisches Potential", (20, 5)),
        ("Zu schwaches optisches Potential", (2, 2)),
        ("Zu steiles optisches Potential", (100000, 0.1))
    ],
    description="optisches Potential",
    style={"description_width": "initial"})

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(params=params_widget)
def make_animation(params):
    t_end = 2
    n_time = 200
    n_max = 4096
    x_max = 15
    k_0 = 15
    v_opt, sigma_opt = params
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max,
                               retstep=True, endpoint=False)
    t_values, dt = np.linspace(0, t_end, n_time+1,
                               retstep=True)
    psi_squared = time_development(
        t_values, dt, x_values, dx, x_max, k_0, v_opt,
        sigma_opt)

    def init():
        line.set_data(x_values, psi_squared[0, :])
        return line,

    def animate(i):
        line.set_data(x_values, psi_squared[i, :])
        return line,

    clear_output()
    fig, ax = plt.subplots()
    line, = ax.plot([], [])
    ax.set_xlim((-x_max, x_max))
    y_max = np.max(psi_squared)
    ax.set_ylim((0, y_max))
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\vert\Psi(x, t)\vert^2$")
    anim = animation.FuncAnimation(fig, animate,
                                   init_func=init,
                                   frames=n_time,
                                   interval=10,
                                   blit=True, repeat=False)
    plt.close()
    display(HTML(anim.to_jshtml()))

## Breite des Wellenpakets

### Berechnung der Standardabweichung des Ortes

Aus der Aufenthaltswahrscheinlichkeit `psi_squared_of_t`, den Ortswerten `x` sowie dem Gitterabstand `dx` wird die Standardabweichung `sigma_of_t` berechnet. Durch Verwendung von NumPy-Arrays ist keine explizite Schleife über die Zeiten erforderlich. Man muss jedoch darauf achten, die Summation entlang der Achse 1 auszuführen.

In [ ]:
def sigma(psi_squared_of_t, x, dx):
    x_mean = np.sum(psi_squared_of_t*x, axis=1) * dx
    x2_mean = np.sum(psi_squared_of_t*x**2, axis=1) * dx
    sigma_of_t = np.sqrt(x2_mean-x_mean**2)
    return sigma_of_t

### Implementierung des Bedienelements und graphische Darstellung der zeitabhängigen Standardabweichung des Ortes

Mit Hilfe des Schiebereglers lässt sich folgender Parameter einstellen:
- `t_end`: Länge des zu betrachtenden Zeitintervalls

In der Funktion `plot_result` sind einige Parameter fest vorgegeben, können aber bei Bedarf dort verändert werden. Es werden `n_time=20` Zeitwerte betrachtet und das räumliche Gitter umfasst `n_max=4096` Gitterpunkte und läuft von `-x_max=-15` bis `x_max=15` in Einheiten der Breite des Anfangszustands.  Die Berechnung erfolgt ohne ein optisches Potential (`v_opt=0`), da sich das Wellenpaket nicht bewegt (`k_0=0`) und die maximale Endzeit `t_end` hinreichend klein sowie das betrachtete Ortsintervall hinreichend groß ist, so dass Randeffekte vernachlässigt werden können. Der Zeitschritt für die Split-Operator-Methode ergibt sich aus `t_end` dividiert durch `n_time`. Die numerischen Daten sind als Punkte dargestellt, während das analytische Ergebnis als durchgezogene Linie gezeigt ist.

In [ ]:
t_end_widget = widgets.FloatSlider(
    value=5, min=0.1, max=10, step=0.1,
    description=r"$t_\text{end}$")

@interact(t_end=t_end_widget)
def plot_result(t_end):
    n_time = 20
    n_max = 4096
    x_max = 15
    k_0 = 0
    v_opt = 0
    sigma_opt = 1
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max,
                               retstep=True, endpoint=False)
    t_values, dt = np.linspace(0, t_end, n_time+1,
                               retstep=True)
    psi_squared_of_t = time_development(
        t_values, dt, x_values, dx, x_max, k_0, v_opt,
        sigma_opt)
    sigma_of_t = sigma(psi_squared_of_t, x_values, dx)

    fig, ax = plt.subplots()
    ax.plot(t_values, sigma_of_t, label="numerisch",
            linestyle="None", marker="o")
    ax.plot(t_values, np.sqrt(1+t_values**2/4),
            label="analytisch")
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\sigma$")
    ax.legend(loc="upper left")